In [8]:
import pandas as pd
import numpy as np
import math

def format_with_uncertainty(value, error, unidad, max_error_digits=2):
    """
    Formatea un valor con su incertidumbre siguiendo las reglas de notación científica.
    
    Args:
        value (float): Valor medido
        error (float): Incertidumbre del valor
        max_error_digits (int): Número máximo de cifras significativas en el error
    
    Returns:
        str: Cadena formateada en notación científica
    """
    if pd.isna(value) or pd.isna(error) or error == 0:
        return f"{value:.2e}"
    
    # Encontrar el orden de magnitud del error
    error_order = math.floor(math.log10(abs(error)))
    
    # Calcular el factor de escala para redondear el error
    scale_factor = 10 ** (error_order - max_error_digits + 1)
    
    # Redondear el error al número deseado de cifras significativas
    rounded_error = round(error / scale_factor) * scale_factor
    
    # Determinar el orden de magnitud común (basado en el valor principal)
    if value != 0:
        value_order = math.floor(math.log10(abs(value)))
    else:
        value_order = error_order
    
    # Escalar ambos valores al mismo orden de magnitud
    common_scale = 10 ** value_order
    scaled_value = value / common_scale
    scaled_error = rounded_error / common_scale
    
    # Determinar el número de decimales necesarios
    if error_order >= value_order:
        decimals = max_error_digits - 1
    else:
        decimals = value_order - error_order + max_error_digits - 1
    
    decimals = max(0, decimals)  # No puede ser negativo
    
    # Formatear el resultado
    if value_order == 0:
        return f"({scaled_value:.{decimals}f} ± {scaled_error:.{decimals}f}) [{unidad}]"
    else:
        return f"({scaled_value:.{decimals}f} ± {scaled_error:.{decimals}f}) × 10^{value_order} [{unidad}]"

def format_dataframe_with_uncertainty(df, value_col, error_col, unidad, cifras_significativas = 2, result_col='formatted'):
    """
    Aplica el formateo con incertidumbre a un DataFrame completo.
    
    Args:
        df (pd.DataFrame): DataFrame con los datos
        value_col (str): Nombre de la columna con los valores
        error_col (str): Nombre de la columna con las incertidumbres
        result_col (str): Nombre de la columna resultado
    
    Returns:
        pd.DataFrame: DataFrame con la columna formateada añadida
    """
    df_result = df.copy()
    df_result[result_col] = df.apply(
        lambda row: format_with_uncertainty(row[value_col], row[error_col], unidad, cifras_significativas), 
        axis=1
    )
    return df_result

def generar_archivo_cientifico(path, distancia_m = 1):
    tiempos_df = tiempos_df = pd.read_csv(path, delimiter= ';', dtype=float, usecols=[0])

    error_tiempo_s = 20e-6
    error_distancia_m = 1e-3 
    velocidad_sonido_m_s = distancia_m / tiempos_df
    error_velocidad_m_s = velocidad_sonido_m_s * np.sqrt((error_distancia_m/distancia_m)**2 + (error_tiempo_s/tiempos_df)**2)

    datos_df = pd.DataFrame()
    datos_df['tiempo'] = tiempos_df
    datos_df['distancia'] = distancia_m
    datos_df['error_distancia'] = error_distancia_m
    datos_df['error_tiempo'] = error_tiempo_s
    datos_df['velocidad'] = velocidad_sonido_m_s
    datos_df['error_velocidad'] = error_velocidad_m_s

    tiempo_formato_df = format_dataframe_with_uncertainty(datos_df, 'tiempo', 'error_tiempo', 's', cifras_significativas=1, result_col='tiempo_cientifico')
    distancia_formato_df = format_dataframe_with_uncertainty(tiempo_formato_df, 'distancia', 'error_distancia', 'm', cifras_significativas=1, result_col='distancia_cientifica')
    datos_formato_df = format_dataframe_with_uncertainty(distancia_formato_df, 'velocidad', 'error_velocidad', 'm/s', result_col='velocidad_cientifica')

    return datos_df, datos_formato_df[['tiempo_cientifico', 'distancia_cientifica',  'velocidad_cientifica']]


In [9]:
velocidad_sonido_100cm_df, velocidad_sonido_cientifica_100cm_df = generar_archivo_cientifico(r'Datos\Tiempos 100cm.txt', distancia_m=2)
velocidad_sonido_80cm_df , velocidad_sonido_cientifica_80cm_df = generar_archivo_cientifico(r'Datos\Tiempos 80cm.txt', distancia_m=1.6)
velocidad_sonido_60cm_df , velocidad_sonido_cientifica_60cm_df = generar_archivo_cientifico(r'Datos\Tiempos 60cm.txt', distancia_m=1.2)
velocidad_sonido_40cm_df , velocidad_sonido_cientifica_40cm_df = generar_archivo_cientifico(r'Datos\Tiempos 40cm.txt', distancia_m=0.8)
velocidad_sonido_20cm_df , velocidad_sonido_cientifica_20cm_df = generar_archivo_cientifico(r'Datos\Tiempos 20cm.txt', distancia_m=0.4)

In [10]:
print(f'velocidad [m/s] = {velocidad_sonido_100cm_df.velocidad.mean()}\nError_velocidad [m/s] = {np.sqrt(velocidad_sonido_100cm_df.error_distancia.max()**2 + velocidad_sonido_100cm_df.error_distancia.std())}')

velocidad [m/s] = 337.2788229406481
Error_velocidad [m/s] = 0.001


In [11]:
print(f'velocidad [m/s] = {velocidad_sonido_80cm_df.velocidad.mean()}\nError_velocidad [m/s] = {np.sqrt(velocidad_sonido_80cm_df.error_distancia.max()**2 + velocidad_sonido_80cm_df.error_distancia.std())}')

velocidad [m/s] = 334.5524789060634
Error_velocidad [m/s] = 0.001


In [12]:
print(f'velocidad [m/s] = {velocidad_sonido_60cm_df.velocidad.mean()}\nError_velocidad [m/s] = {np.sqrt(velocidad_sonido_60cm_df.error_distancia.max()**2 + velocidad_sonido_60cm_df.error_distancia.std())}')

velocidad [m/s] = 316.7992553601711
Error_velocidad [m/s] = 0.001


In [13]:
print(f'velocidad [m/s] = {velocidad_sonido_40cm_df.velocidad.mean()}\nError_velocidad [m/s] = {np.sqrt(velocidad_sonido_40cm_df.error_distancia.max()**2 + velocidad_sonido_40cm_df.error_distancia.std())}')

velocidad [m/s] = 303.86329663994223
Error_velocidad [m/s] = 0.001


In [ ]:
print(f'velocidad [m/s] = {velocidad_sonido_20cm_df.velocidad.mean()}\nError_velocidad [m/s] = {np.sqrt(velocidad_sonido_20cm_df.error_distancia.max()**2 + velocidad_sonido_20cm_df.error_distancia.std())}')

velocidad [m/s] = 345.8471158863136
Error_velocidad [m/s] = 0.001


In [20]:
velocidad_sonido_promedio = np.array([velocidad_sonido_100cm_df.velocidad.mean(),
velocidad_sonido_80cm_df.velocidad.mean(),
velocidad_sonido_60cm_df.velocidad.mean(),
velocidad_sonido_40cm_df.velocidad.mean(),
velocidad_sonido_20cm_df.velocidad.mean()]).mean()

velocidad_sonido_std = np.array([velocidad_sonido_100cm_df.velocidad.mean(),
velocidad_sonido_80cm_df.velocidad.mean(),
velocidad_sonido_60cm_df.velocidad.mean(),
velocidad_sonido_40cm_df.velocidad.mean(),
velocidad_sonido_20cm_df.velocidad.mean()]).std()

print(f'velocidad [m/s] = {velocidad_sonido_promedio}\nError_velocidad [m/s] = {np.sqrt(velocidad_sonido_std**2 + velocidad_sonido_20cm_df.error_distancia.std())}')

velocidad [m/s] = 327.6681939466277
Error_velocidad [m/s] = 15.19893242126313
